Identifying spam in popular YouTube videos
comments with LSTM - Keras

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

In [ ]:
%cd '/content/gdrive/My Drive/LDS8_DeepLearning/Practice/Chapter7_NLP/'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embe
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
comments_df_list = []
comments_file = ['data/Youtube01-Psy.csv',
                  'data/Youtube02-KatyPerry.csv',
                  'data/Youtube03-LMFAO.csv',
                  'data/Youtube04-Eminem.csv',
                  'data/Youtube05-Shakira.csv']
for f in comments_file:
  print(f)
  df = pd.read_csv(f,header=0)
  comments_df_list.append(df)
comments_df = pd.concat(comments_df_list)
# comments_df = comments_df.sample(frac=1.0)
print(comments_df.shape)
comments_df.head(5)

In [ ]:
comments_df.columns

In [ ]:
# https://www.kaggle.com/kredy10/simple-lstm-for-text-classification
df = comments_df.drop(['COMMENT_ID', 'AUTHOR', 'DATE'], axis = 1)
df.info()

In [ ]:
df['V1'] = df['CLASS'].map(lambda x: 'ham' if x==0 else 'spam')
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.countplot(df.CLASS)
plt.xlabel('Label')
plt.title('Number of ham-0 and spam-1 content')

In [ ]:
X = df.CONTENT # input
Y = df.CLASS # output

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

Process the data

Tokenize the data and convert the text to sequences.

Add padding to ensure that all the sequences have the same shape.

There are many ways of taking the max_len and here an arbitrary length of 150 is chosen.

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train) # 3 => 3 word IDs, 5 => 5 wordIDs
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

In [ ]:
print(sequences[:2])

In [ ]:
sequences_matrix[:2]

#RNN

In [ ]:
def RNN():
  inputs = Input(name='inputs',shape=[max_len])
  layer = Embedding(max_words, 50, input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256,name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1,name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

In [ ]:
# model = RNN()
model = Sequential()
model.add(Embedding(max_words, 50))
model.add(LSTM(64))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# model = RNN()
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
model.summary()

CALCULATE embedling layer: input * output = 1000*50 = 50000

lstm layer = [(num_unit + inputdims)num_unit]4 = (64+50+1)644 = 29440

dense layer = 64*256 + 256 = 16640

dropout = 0

dense output layer = 256*1 + 1 = 257


In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

In [ ]:
plot_model(model, to_file='LSTM_model.png', show_shapes=True)
Image(filename='LSTM_model.png')

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(sequences_matrix,Y_train, # train
                    batch_size=128,
                    epochs=50,
                    validation_split=0.2,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

In [ ]:
# test
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

In [ ]:
print('Test set\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
#Save the result
from tensorflow.keras.models import load_model
# Creates a HDF5 file 'my_model.h5'
model.save('LSTM_spam_model.h5')

Make new predictions

In [ ]:
type(X_test)

In [ ]:
test_sequences_new = tok.texts_to_sequences(X_new)
test_sequences_matrix_new = sequence.pad_sequences(test_sequences_new,maxlen=max_len)

In [ ]:
X_new

In [ ]:
test_sequences_matrix_new

In [ ]:
model.predict(test_sequences_matrix_new)